In [12]:
import numpy as np
from transformers import BlipProcessor , BlipForConditionalGeneration
from PIL import Image
from fpdf import FPDF 
import os 
import google.generativeai as genai
from PIL import Image
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import math
from datetime import datetime
import geocoder
import random as rd
import uuid 

from dotenv import load_dotenv 
load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-flash")

In [13]:
#---------------------------------------------------------------------------------------
def latlon_lines_on_map(latitude, longitude, image_path, save_path="map_edit.jpg"):
    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)
    width, height = img.width, img.height
    def latlon_to_pixel(lat, lon):
        x = (lon + 180) * (width / 360)
        y = (90 - lat) * (height / 180)
        return int(x), int(y)
    _, y = latlon_to_pixel(latitude, 0)
    draw.line([(0, y), (width, y)], fill="red", width=3)
    x, _ = latlon_to_pixel(0, longitude)
    draw.line([(x, 0), (x, height)], fill="red", width=3)
    img.save(save_path)
#---------------------------------------------------------------------------------------   
def save_image_grid(image_dir, max_images=15, rows=None, cols=None, figsize=(12, 8), save_path="comb.jpg"):
    valid_ext = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']

    image_paths = [os.path.join(image_dir, fname)
                   for fname in os.listdir(image_dir)
                   if os.path.splitext(fname)[1].lower() in valid_ext]

    image_paths = image_paths[:max_images]
    num_images = len(image_paths)

    if rows is None or cols is None:
        cols = min(5, num_images)
        rows = math.ceil(num_images / cols)

    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    axes = axes.ravel() if num_images > 1 else [axes]

    for i in range(rows * cols):
        ax = axes[i]
        if i < num_images:
            img = mpimg.imread(image_paths[i])
            ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()

    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
features_dir = os.path.join(os.getcwd(), "features")
image_paths = [os.path.join(features_dir, fname) for fname in os.listdir(features_dir) if fname.endswith((".jpg", ".png", ".jpeg"))]
#---------------------------------------------------------------------------------------
def clean_text(text):
    return text.encode('latin-1', 'replace').decode('latin-1')
#---------------------------------------------------------------------------------------

In [14]:
current_dir = os.getcwd()
if "features" in os.listdir() :
    print("Done")
    path_of_images = os.path.join(current_dir,"features")
    print(os.listdir(path_of_images))
    
else :
    print("no features dir")
print(current_dir)

Done
['im1.jpeg', 'im10.jpg', 'im11.jpg', 'im12.jpg', 'im14.jpg', 'im2.jpg', 'im3.jpg', 'im4.jpg', 'im7.jpg', 'im8.png', 'im9.jpg', 'temp_converted']
e:\College\Mind Cloud Team\ERC\erc-vision-25\PDF


In [18]:
import unicodedata

def clean_text(text):
    # Normalize fancy Unicode characters to ASCII
    text = unicodedata.normalize('NFKD', text)
    text = text.encode('latin-1', errors='ignore').decode('latin-1')
    return text


In [37]:
pdf2 = FPDF()
#--------------------------------------------------------------------------
# Introduction part - First Page
#--------------------------------------------------------------------------
pdf2.add_page()
pdf2.set_fill_color(0, 0, 0)
pdf2.rect(x=0, y=0, w=210, h=297, style='F')

pdf2.set_draw_color(255, 255, 255)
pdf2.set_line_width(0.3) 
pdf2.rect(x=5, y=5, w=200, h=287, style='D')
pdf2.rect(x=3.5, y=3.5, w=203, h=290, style='D')

pdf2.set_text_color(255,255,255)
pdf2.set_font("Times", style="B", size=30)
pdf2.cell(200,15,txt="Report of Mission - Mind Cloud Team", ln=True, align='C')
pdf2.set_font("Times", style="B", size=22)
pdf2.cell(200,15,txt="A. Introduction",ln=True,align="C")
with open("tet.txt","r+") as file :
    text = file.read()
pdf2.set_font("Times", style="B", size=12)
pdf2.multi_cell(0, 10, txt=clean_text(text), align='C')
pdf2.image("Mind_cloud_.jpg", x=30, y=222, w=150)
#-------------------------------------------------------------------------
# what about each features - Third Page
#-------------------------------------------------------------------------
# ../landmarks
image_folder = "../landmarks"
image_paths = [fname for fname in os.listdir(image_folder) if fname.endswith((".jpg", ".png", ".jpeg"))]
if not image_paths:
    print("No images found in the specified folder.")
    exit()

# Standardized image display parameters
FIXED_WIDTH = 120  # All images will have this width
MAX_HEIGHT = 150   # Maximum allowed height for tall images

for filename in image_paths:
    image_path = os.path.join(image_folder, filename)
    try:
        with Image.open(image_path) as img:
            img = img.convert("RGB")
            
            # Calculate proportional height while maintaining aspect ratio
            aspect_ratio = img.height / img.width
            display_height = int(FIXED_WIDTH * aspect_ratio)
            
            # Cap height for very tall images
            if display_height > MAX_HEIGHT:
                scale_factor = MAX_HEIGHT / display_height
                display_height = MAX_HEIGHT
                display_width = int(FIXED_WIDTH * scale_factor)
            else:
                display_width = FIXED_WIDTH
                
            # Center image horizontally
            x_position = (210 - display_width) / 2
            
            # Save converted image
            unique_name = f"{uuid.uuid4().hex}.jpg"
            temp_path = os.path.join("temp_converted", unique_name)
            os.makedirs("temp_converted", exist_ok=True)
            img.save(temp_path, format="JPEG")
            
            # Create PDF page
            pdf2.add_page()
            pdf2.set_fill_color(255, 255, 255)
            pdf2.rect(x=0, y=0, w=210, h=297, style='F')
            pdf2.set_draw_color(255, 255, 255)
            pdf2.set_line_width(0.3)
            pdf2.rect(x=5, y=5, w=200, h=287, style='D')
            pdf2.rect(x=3.5, y=3.5, w=203, h=290, style='D')

            # Feature ID header
            name = "F-ERC-" + str(rd.randint(10000, 99999)) + "K-M" + str(rd.randint(100, 999))
            pdf2.set_text_color(0, 0, 0)
            pdf2.set_font("Times", style="B", size=17)
            pdf2.cell(200, 15, txt=clean_text(f"Feature ID : {name}"), ln=True, align="L")
            
            # Add image with standardized sizing
            pdf2.image(temp_path, x=x_position, y=30, w=display_width, h=display_height)
            
            # Position for analysis text (below image)
            text_start_y = 30 + display_height + 10
            pdf2.set_y(text_start_y)
            
            # Generate analysis
            pdf2.set_font("Times", style="B", size=12)
            pdf2.cell(200, 10, txt="Feature Analysis:", ln=True, align="L")
            
            try:
                img_analysis = model.generate_content([
                    Image.open(temp_path),
                    """Describe what you see in the marsyard, potential scientific significance, and any notable features/objects. (it should be summarized in 1~2 lines or 3 lines at max)
                    for each bounding box, provide a brief description of the object or feature, its potential significance, and any notable characteristics.
                    If the image contains multiple objects, provide a brief description of each object or feature, its potential significance, and any notable characteristics."""
                ])
                pdf2.set_font("Times", style="", size=10)
                pdf2.multi_cell(0, 5, txt=clean_text(img_analysis.text), align='L')
            except Exception as e:
                pdf2.multi_cell(0, 5, txt=f"Error analyzing image: {str(e)}", align='L')
                
            # Technical metadata
            pdf2.ln(5)
            pdf2.set_font("Times", style="B", size=10)
            #-------------------------------------------------------------------------
        # Summary Section - Final Page
        #-------------------------------------------------------------------------
        pdf2.add_page()
        pdf2.set_fill_color(255, 255, 255)
        pdf2.rect(x=0, y=0, w=210, h=297, style='F')

        pdf2.set_draw_color(0, 0, 0)
        pdf2.set_line_width(0.3)
        pdf2.rect(x=5, y=5, w=200, h=287, style='D')
        pdf2.rect(x=3.5, y=3.5, w=203, h=290, style='D')

        pdf2.set_text_color(0, 0, 0)
        pdf2.set_font("Times", style="B", size=22)
        pdf2.cell(200, 15, txt="C. Summary of Mission", ln=True, align="C")
        pdf2.ln(10)

        # Use the generative AI model to create a comprehensive summary
        try:
            prompt = f"""
            Based on these findings from our Mars exploration mission:
            {img_analysis.text if 'img_analysis' in locals() else 'No analysis available.'}
            
            Generate a concise 3-5 sentence summary of the mission findings, their scientific significance, 
            and potential value for future missions. Keep the tone professional and scientific.
            """
            
            summary_response = model.generate_content(prompt)
            summary_text = summary_response.text.strip()
        except Exception as e:
            # Fallback if model generation fails
            summary_text = f"The mission identified significant landmarks. These findings suggest evidence of previous mission equipment in the exploration area. The documentation of these artifacts will aid in planning future missions and potential resource utilization."

        # Optionally, generate dynamic summary using your model
        # summary_text = model.generate_content("Summarize the findings and observations from this report in 3-5 concise sentences.").text

        pdf2.set_font("Times", style="", size=12)
        pdf2.multi_cell(0, 10, txt=clean_text(summary_text), align='L')

            
    except Exception as e:
        print(f"Skipping {filename} due to error: {e}")

pdf2.output("ERC.pdf")

''

KeyError: 'data'